# Feature Engineering


Author: Jasmine Qin  
Date: 2020-06-29

In [1]:
# Basics
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import time
import re
import json
from collections import defaultdict, Counter

# Shapely
from shapely.ops import nearest_points
from shapely.geometry import Point, Polygon
import shapely.speedups

# For nearby business
from functools import partial
import pyproj
from shapely.ops import transform
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
shapely.speedups.enable()

In [3]:
combined_train = pd.read_csv('../../data/processed/04_combined_train.csv',
                    low_memory=False)
combined_validation = pd.read_csv('../../data/processed/04_combined_validate.csv',
                         low_memory=False)
combined_test = pd.read_csv('../../data/processed/04_combined_test.csv',
                         low_memory=False)

In [4]:
all_licence = pd.concat([combined_train, combined_validation, combined_test])
all_licence = all_licence[~(all_licence.FOLDERYEAR == 2020)]
all_licence = all_licence[all_licence.Geom.notnull()]
all_licence = all_licence[['LicenceRSN', 'FOLDERYEAR', 'BusinessType', 'Geom']]

all_licence['lat'] = all_licence.Geom.apply(lambda p: json.loads(p)['coordinates'][1])
all_licence['lon'] = all_licence.Geom.apply(lambda p: json.loads(p)['coordinates'][0])

In [5]:
all_licence.shape

(453989, 6)

## Nearby business - train set

In [6]:
# return buffer region
def buffer_polygon(lat, lon, m):

    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    
    buffer = transform(project, Point(0, 0).buffer(m)).exterior.coords[:]
    
    return Polygon([[p[0], p[1]] for p in buffer])

In [ ]:
def business_lookup(df):
    business_lookup = defaultdict(dict)
    for y in df.FOLDERYEAR.unique():
        for i in df.BusinessType.unique():
            df_y_i = df[(df.FOLDERYEAR == y) & (df.BusinessType == i)]
            points = df_y_i.point
            for index, row in df_y_i.iterrows():
                business_lookup[y][row.business_id] = np.asarray(
                    [row.polygon.contains(i) for i in points]).sum()-1

    business_lookup = {(outerKey, innerKey): values for outerKey, innerDict in business_lookup.items()
                       for innerKey, values in innerDict.items()}
    business_lookup = pd.DataFrame(business_lookup, index=[
        'nearest_business_count']).T.reset_index().rename(
        columns={'level_0': 'FOLDERYEAR',
                 'level_1': 'business_id'})

    df = df.merge(
        business_lookup,
        how='left',
        left_on=['FOLDERYEAR', 'business_id'],
        right_on=['FOLDERYEAR', 'business_id'])

    return df

In [13]:
dict_all_years = defaultdict()

In [14]:
df_1997 = all_licence[all_licence.FOLDERYEAR == 1997]
df_1997['point'] = df_1997.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_1997['polygon'] = df_1997.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_1997.BusinessType.unique():
    df_i = df_1997[df_1997.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [34]:
df_1998 = all_licence[all_licence.FOLDERYEAR == 1998]
df_1998['point'] = df_1998.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_1998['polygon'] = df_1998.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_1998.BusinessType.unique():
    df_i = df_1998[df_1998.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
df_1999 = all_licence[all_licence.FOLDERYEAR == 1999]
df_1999['point'] = df_1999.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_1999['polygon'] = df_1999.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_1999.BusinessType.unique():
    df_i = df_1999[df_1999.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [35]:
df_2000 = all_licence[all_licence.FOLDERYEAR == 2000]
df_2000['point'] = df_2000.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2000['polygon'] = df_2000.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2000.BusinessType.unique():
    df_i = df_2000[df_2000.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [39]:
print(len(dict_all_years))
print(pd.concat([df_1997, df_1998, df_1999, df_2000]).shape)

63725
(63736, 8)


In [40]:
df_2001 = all_licence[all_licence.FOLDERYEAR == 2001]
df_2001['point'] = df_2001.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2001['polygon'] = df_2001.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2001.BusinessType.unique():
    df_i = df_2001[df_2001.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [41]:
df_2002 = all_licence[all_licence.FOLDERYEAR == 2002]
df_2002['point'] = df_2002.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2002['polygon'] = df_2002.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2002.BusinessType.unique():
    df_i = df_2002[df_2002.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [42]:
df_2003 = all_licence[all_licence.FOLDERYEAR == 2003]
df_2003['point'] = df_2003.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2003['polygon'] = df_2003.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2003.BusinessType.unique():
    df_i = df_2003[df_2003.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [43]:
print(len(dict_all_years))
print(pd.concat([df_1997, df_1998, df_1999, df_2000, df_2001, df_2002, df_2003]).shape)

120253
(120265, 8)


In [45]:
df_2004 = all_licence[all_licence.FOLDERYEAR == 2004]
df_2004['point'] = df_2004.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2004['polygon'] = df_2004.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2004.BusinessType.unique():
    df_i = df_2004[df_2004.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [46]:
df_2005 = all_licence[all_licence.FOLDERYEAR == 2005]
df_2005['point'] = df_2005.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2005['polygon'] = df_2005.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2005.BusinessType.unique():
    df_i = df_2005[df_2005.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [47]:
df_2006 = all_licence[all_licence.FOLDERYEAR == 2006]
df_2006['point'] = df_2006.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2006['polygon'] = df_2006.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2006.BusinessType.unique():
    df_i = df_2006[df_2006.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [49]:
print(len(dict_all_years))
print(pd.concat([df_1997, df_1998, df_1999, df_2000, 
                 df_2001, df_2002, df_2003, df_2004,
                 df_2005, df_2006]).shape)

176950
(176963, 8)


In [50]:
df_2007 = all_licence[all_licence.FOLDERYEAR == 2007]
df_2007['point'] = df_2007.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2007['polygon'] = df_2007.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2007.BusinessType.unique():
    df_i = df_2007[df_2007.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [51]:
df_2008 = all_licence[all_licence.FOLDERYEAR == 2008]
df_2008['point'] = df_2008.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2008['polygon'] = df_2008.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2008.BusinessType.unique():
    df_i = df_2008[df_2008.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [52]:
df_2009 = all_licence[all_licence.FOLDERYEAR == 2009]
df_2009['point'] = df_2009.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2009['polygon'] = df_2009.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2009.BusinessType.unique():
    df_i = df_2009[df_2009.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [53]:
print(len(dict_all_years))
print(pd.concat([df_1997, df_1998, df_1999, df_2000, 
                 df_2001, df_2002, df_2003, df_2004,
                 df_2005, df_2006, df_2007, df_2008,
                 df_2009]).shape)

236579
(236593, 8)


In [54]:
df_2010 = all_licence[all_licence.FOLDERYEAR == 2010]
df_2010['point'] = df_2010.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2010['polygon'] = df_2010.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2010.BusinessType.unique():
    df_i = df_2010[df_2010.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [55]:
df_2011 = all_licence[all_licence.FOLDERYEAR == 2011]
df_2011['point'] = df_2011.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2011['polygon'] = df_2011.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2011.BusinessType.unique():
    df_i = df_2011[df_2011.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [56]:
df_2012 = all_licence[all_licence.FOLDERYEAR == 2012]
df_2012['point'] = df_2012.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2012['polygon'] = df_2012.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2012.BusinessType.unique():
    df_i = df_2012[df_2012.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [57]:
print(len(dict_all_years))
print(pd.concat([df_1997, df_1998, df_1999, df_2000, 
                 df_2001, df_2002, df_2003, df_2004,
                 df_2005, df_2006, df_2007, df_2008,
                 df_2009, df_2010, df_2011, df_2012]).shape)

299501
(299515, 8)


In [58]:
df_2013 = all_licence[all_licence.FOLDERYEAR == 2013]
df_2013['point'] = df_2013.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2013['polygon'] = df_2013.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2013.BusinessType.unique():
    df_i = df_2013[df_2013.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [59]:
df_2014 = all_licence[all_licence.FOLDERYEAR == 2014]
df_2014['point'] = df_2014.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2014['polygon'] = df_2014.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2014.BusinessType.unique():
    df_i = df_2014[df_2014.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [66]:
df_2015 = all_licence[all_licence.FOLDERYEAR == 2015]
df_2015['point'] = df_2015.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2015['polygon'] = df_2015.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2015.BusinessType.unique():
    df_i = df_2015[df_2015.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [61]:
df_2016 = all_licence[all_licence.FOLDERYEAR == 2016]
df_2016['point'] = df_2016.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2016['polygon'] = df_2016.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2016.BusinessType.unique():
    df_i = df_2016[df_2016.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [62]:
df_2017 = all_licence[all_licence.FOLDERYEAR == 2017]
df_2017['point'] = df_2017.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2017['polygon'] = df_2017.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2017.BusinessType.unique():
    df_i = df_2017[df_2017.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [63]:
df_2018 = all_licence[all_licence.FOLDERYEAR == 2018]
df_2018['point'] = df_2018.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2018['polygon'] = df_2018.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2018.BusinessType.unique():
    df_i = df_2018[df_2018.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [64]:
df_2019 = all_licence[all_licence.FOLDERYEAR == 2019]
df_2019['point'] = df_2019.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2019['polygon'] = df_2019.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

for i in df_2019.BusinessType.unique():
    df_i = df_2019[df_2019.BusinessType == i]
    points = df_i.point

    for index, row in df_i.iterrows():
        dict_all_years[row.LicenceRSN] = np.asarray(
            [row.polygon.contains(p) for p in points]).sum()-1

In [67]:
print(len(dict_all_years))
print(pd.concat([df_1997, df_1998, df_1999, df_2000, 
                 df_2001, df_2002, df_2003, df_2004,
                 df_2005, df_2006, df_2007, df_2008,
                 df_2009, df_2010, df_2011, df_2012,
                 df_2013, df_2014, df_2015, df_2016,
                 df_2017, df_2018, df_2019]).shape)

453973
(453989, 8)


In [70]:
pd.DataFrame(dict_all_years, index=[
    'nearest_business_count']).T.reset_index().rename(
    columns={'index': 'LicenceRSN'}).to_csv("nearby_business.csv", index=False)


# VIZ

In [9]:
# return buffer region
def buffer_polygon(lat, lon, m):

    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    
    buffer = transform(project, Point(0, 0).buffer(m)).exterior.coords[:]
    
    return [[p[0], p[1]] for p in buffer]

df_2019 = all_licence[all_licence.FOLDERYEAR == 2019]
df_2019['point'] = df_2019.apply(lambda p: Point(p.lon, p.lat), axis=1)
df_2019['polygon'] = df_2019.apply(
    lambda p: buffer_polygon(p.lat, p.lon, 200), axis=1)

In [8]:
df_2019[df_2019.BusinessType == "Restaurant Class 1"]

,LicenceRSN,FOLDERYEAR,BusinessType,Geom,lat,lon,point,polygon
598,3274259,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.1150384...",49.282881,-123.115038,POINT (-123.115038457565 49.2828811521773),POLYGON ((-123.1122895626764 49.28288111948046...
3005,3274517,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.0700302...",49.267383,-123.070030,POINT (-123.070030204642 49.2673831280343),POLYGON ((-123.0672821708492 49.26738309535521...
9084,3274634,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.1177787...",49.262946,-123.117779,POINT (-123.117778739981 49.2629459727502),POLYGON ((-123.1150309525872 49.26294594007621...
13066,3275064,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.0699786...",49.272534,-123.069979,POINT (-123.069978672124 49.2725341405952),"POLYGON ((-123.0672303522148 49.2725341079102,..."
19277,3274883,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.0571190...",49.261581,-123.057119,POINT (-123.057119059475 49.261580734179),POLYGON ((-123.0543713478819 49.26158070150657...
...,...,...,...,...,...,...,...,...
132985,3274697,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.1215131...",49.275327,-123.121513,POINT (-123.121513172254 49.2753265683385),POLYGON ((-123.1187646972033 49.27532653565031...
133873,3273802,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.2086421...",49.263536,-123.208642,POINT (-123.208642168507 49.2635361848197),POLYGON ((-123.2058943483417 49.26353615214504...
134050,3274908,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.0538490...",49.225386,-123.053849,POINT (-123.053849078656 49.2253858085065),POLYGON ((-123.0511033745764 49.22538577587552...
134695,3275518,2019,Restaurant Class 1,"{""type"": ""Point"", ""coordinates"": [-123.0640985...",49.281458,-123.064099,POINT (-123.064098571573 49.2814576889248),POLYGON ((-123.0613497558053 49.28145765622959...


In [11]:
point = df_2019[df_2019.LicenceRSN == 3274259].point
polygon = df_2019[df_2019.LicenceRSN == 3274259].polygon
points = df_2019.point

In [37]:
x=[i[0] for i in polygon.values[0]]
y=[i[1] for i in polygon.values[0]]
x.append(df_2019[df_2019.LicenceRSN == 3274259].lat.values[0])
y.append(df_2019[df_2019.LicenceRSN == 3274259].lon.values[0])
color=['black']*len(x)

for p in points:
    if Polygon(polygon.values[0]).contains(p):
        x.append(p.x)
        y.append(p.y)
        if p.x == df_2019[df_2019.LicenceRSN == 3274259].lon.values[0] and p.y == df_2019[df_2019.LicenceRSN == 3274259].lat.values[0]:
            color.append("red")
        else:
            color.append("blue")

new_df=pd.DataFrame({"x":x, "y":y, "color":color})

# Basics
import pandas as pd
import geopandas as gpd
import numpy as np
import json

# Plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot

mapbox_access_token = "pk.eyJ1IjoiamFzbWluZXF5aiIsImEiOiJja2Fyc2toN2Ewb3FxMnJsZzhuN3N3azk2In0.SJcixuEa_agNUDz7fFYDEg"
px.set_mapbox_access_token(mapbox_access_token)

scatter = px.scatter_mapbox(new_df,
                            lat="y",
                            lon="x",
                            zoom=16,
                            color_discrete_sequence=[new_df["color"]])

scatter.update_layout(height=800,
                      width=1000,
                      mapbox_center={"lat": 49.250, "lon": -123.121})

scatter.show()